In [2]:
import os
import sys
import random
from glob import glob
from copy import deepcopy
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm

from syotools.models import Camera, Telescope, Spectrograph, PhotometricExposure, SpectrographicExposure
from syotools.utils.jsonunit import str_jsunit

# enable use of local modules
repo_dir = '/grad/bwedig/mejiro'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

from mejiro.instruments.hwo import HWO
from mejiro.synthetic_image import SyntheticImage
from mejiro.lenses.test import SampleStrongLens
from mejiro.exposure import Exposure
from mejiro.utils import util

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

Warning, 5 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 3590 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 293 of 2920 bins contained negative fluxes; they have been set to zero.
Warning, 5 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 3590 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 293 of 2920 bins contained negative fluxes; they have been set to zero.


In [3]:
hwo = HWO()
pprint(hwo.versions)

{'syotools': '1.1.13'}


In [4]:
pprint(hwo.dark_current)

{'B': <Quantity 0.002 electron / (pix s)>,
 'FUV': <Quantity 0.002 electron / (pix s)>,
 'H': <Quantity 0.002 electron / (pix s)>,
 'I': <Quantity 0.002 electron / (pix s)>,
 'J': <Quantity 0.002 electron / (pix s)>,
 'K': <Quantity 0.002 electron / (pix s)>,
 'NUV': <Quantity 0.002 electron / (pix s)>,
 'R': <Quantity 0.002 electron / (pix s)>,
 'U': <Quantity 0.002 electron / (pix s)>,
 'V': <Quantity 0.002 electron / (pix s)>}


In [5]:
hwo.get_dark_current('FUV')

<Quantity 0.002 electron / (pix s)>

In [6]:
camera, telescope = Camera(), Telescope()
telescope.set_from_json('EAC1')
telescope.add_camera(camera)

In [ ]:
flux_zp = {band: bp for band, bp in zip(hwo.bands, camera.recover('ab_zeropoint'))}
pprint(flux_zp)

{'B': <Quantity 11599.78947368 ph / (nm s cm2)>,
 'FUV': <Quantity 24488.44444444 ph / (nm s cm2)>,
 'H': <Quantity 3443.6875 ph / (nm s cm2)>,
 'I': <Quantity 6482.23529412 ph / (nm s cm2)>,
 'J': <Quantity 4372.93650794 ph / (nm s cm2)>,
 'K': <Quantity 2481.93693694 ph / (nm s cm2)>,
 'NUV': <Quantity 20036. ph / (nm s cm2)>,
 'R': <Quantity 7109.5483871 ph / (nm s cm2)>,
 'U': <Quantity 16398.51190476 ph / (nm s cm2)>,
 'V': <Quantity 9092.24422442 ph / (nm s cm2)>}


In [16]:
bandwidth = {band: bp for band, bp in zip(hwo.bands, camera.recover('derived_bandpass'))}
pprint(bandwidth)

{'B': 95.0,
 'FUV': 45.0,
 'H': 320.0,
 'I': 170.0,
 'J': 252.0,
 'K': 444.0,
 'NUV': 55.0,
 'R': 155.0,
 'U': 67.2,
 'V': 121.2}


In [11]:
from astropy import units as u

aperture_cm = telescope.recover('aperture').to(u.cm).value
print(aperture_cm)

720.0


In [20]:
{band: (-1 / 0.4) * np.log10(4 / (np.pi * flux_zp[band].value * (aperture_cm ** 2) * bandwidth[band])) for band in hwo.bands}

{'FUV': 29.12982146685107,
 'NUV': 29.12982146685107,
 'U': 29.12982146685107,
 'B': 29.12982146685107,
 'V': 29.12982146685107,
 'R': 29.12982146685107,
 'I': 29.12982146685107,
 'J': 29.12982146685107,
 'H': 29.12982146685107,
 'K': 29.12982146685107}

In [13]:
dark_current = camera.recover('dark_current')

In [7]:
print(type(dark_current))

<class 'list'>


In [19]:
from astropy.units import Quantity

dark_current_q = Quantity(dark_current[0], dark_current[1])
print(dark_current_q)
print(dark_current_q.value)
print(dark_current_q.unit)

[0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002] electron / (pix s)
[0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002]
electron / (pix s)


In [9]:
print(dark_current[1])

electron / (pix s)


In [17]:
print(type(camera.return_quantity('dark_current')))

return_quantity [<syotools.models.camera.Camera object at 0x7fcfe845c6a0>, 'dark_current']
[[0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002], 'electron / (pix s)']
<class 'list'>


In [19]:
print(type(camera.recover('pixel_size')))

<class 'astropy.units.quantity.Quantity'>


In [18]:
print(type(camera.return_quantity('pixel_size')))

return_quantity [<syotools.models.camera.Camera object at 0x7ff762ea76a0>, 'pixel_size']
[0.00572958 0.00572958 0.00572958 0.00572958 0.00572958 0.00572958
 0.00572958 0.01718873 0.01718873 0.01718873] arcsec / pix
<class 'astropy.units.quantity.Quantity'>


In [9]:
for attr in telescope._tracked_attributes:
    print('{}: {}'.format(attr, str_jsunit(getattr(telescope, attr))))

name: HWO-EAC-1
aperture: 7.2 m
temperature: 270.0 K
ota_emissivity: [0.24, '']
diff_limit_wavelength: [500.0, 'nm']
unobscured_fraction: 1.0


In [10]:
telescope.diff_limit_fwhm

<Quantity 0.01475366 arcsec>

In [11]:
telescope.effective_aperture

<Quantity 7.2 m>

In [12]:
for attr in camera._tracked_attributes:
    print('{}: {}'.format(attr, str_jsunit(getattr(camera,attr))))

name: HRI
pivotwave: [[225.0, 275.0, 336.0, 475.0, 606.0, 775.0, 850.0, 1260.0, 1600.0, 2220.0], 'nm']
bandnames: ['FUV', 'NUV', 'U', 'B', 'V', 'R', 'I', 'J', 'H', 'K']
channels: [[0, 1], [2, 3, 4, 5, 6], [7, 8, 9]]
fiducials: [[400.0, 400.0, 1200.0], 'nm']
total_qe: [[0.1, 0.112, 0.175, 0.211, 0.211, 0.211, 0.145, 0.342, 0.342, 0.335], 'electron / ph']
ap_corr: [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], '']
bandpass_r: [[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0], '']
dark_current: [[0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002], 'electron / (pix s)']
detector_rn: [[2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5], 'electron(1/2) / pix(1/2)']
sky_sigma: [[24.56, 26.27, 23.38, 22.8, 22.41, 22.2, 22.11, 22.32, 22.53, 22.88], '']


In [13]:
camera.ab_zeropoint

<Quantity [24488.44444444, 20036.        , 16398.51190476, 11599.78947368,
            9092.24422442,  7109.5483871 ,  6482.23529412,  4372.93650794,
            3443.6875    ,  2481.93693694] ph / (nm s cm2)>

In [14]:
print(camera.recover('dark_current'))
print(type(camera.recover('dark_current')))

[[0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002], 'electron / (pix s)']
<class 'list'>


In [15]:
from syotools.models import PhotometricExposure

phot_exp = PhotometricExposure()

for attr in phot_exp._tracked_attributes:
    print('{}: {}'.format(attr, str_jsunit(getattr(phot_exp, attr))))

n_exp: 3
exptime: [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'hr']
snr: [[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], '']
magnitude: [0.] mag(AB)
unknown: snr


In [ ]:
ab_magnitudes = 25
exposure_in_hours = 1

diff_limit = 1.22*(500.*0.000000001)*206264.8062/telescope.aperture
print(diff_limit)

source_magnitudes = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]) * ab_magnitudes

sky_brightness = np.array([23.807, 25.517, 22.627, 22.307, 21.917, 22.257, 21.757, 21.567, 22.417, 22.537])
fwhm_psf = 1.22 * camera.pivotwave * 0.000000001 * 206264.8062 / telescope.aperture
fwhm_psf[fwhm_psf < diff_limit] = fwhm_psf[fwhm_psf < diff_limit] * 0.0 + diff_limit

sn_box = np.round(3. * fwhm_psf / camera.pixel_size)

number_of_exposures = np.array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

desired_exposure_time = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]) * exposure_in_hours * 3600.

time_per_exposure = desired_exposure_time / number_of_exposures

signal_counts = camera.total_qe * desired_exposure_time * camera.ab_zeropoint * camera.aperture_correction * (np.pi) / 4. * \
    (telescope.aperture  * 100.0)**2 * camera.derived_bandpass * 10.**(-0.4*ab_magnitudes)

shot_noise_in_signal = signal_counts ** 0.5

sky_counts = camera.total_qe * desired_exposure_time * camera.ab_zeropoint * np.pi / 4. * (telescope.aperture  * 100.0)**2 * \
    camera.derived_bandpass * 10.**(-0.4*sky_brightness) * (camera.pixel_size * sn_box)**2

shot_noise_in_sky = sky_counts ** 0.5

read_noise = camera.detector_read_noise * sn_box * number_of_exposures**0.5

dark_noise = sn_box * (camera.dark_current * desired_exposure_time)**0.5

# thermal_counts = desired_exposure_time * C_thermal(telescope, camera.pivotwave, camera.bandpass_r, sn_box**2, camera.pixel_size, camera.total_qe)

0.0174752127475 1 / m


TypeError: can't multiply sequence by non-int of type 'float'